# Метод опорных векторов

In [29]:
import cv2
from sklearn.svm import LinearSVC
from imutils import paths
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

### Сортировка названий картинок 

In [8]:
imagePaths = sorted(list(paths.list_images("train_task\\train")))

In [44]:
#print(imagePaths)

### Получение гистограмм изображений - характеристик распределения интенсивности изображения, кодировка котов нулями и собак 1

In [19]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def classification(image_path: list[str]) -> list[int]:
    y = []
    for image in imagePaths:
        if ('cat' in image):
            y.append(0)
        elif ('dog' in image):
            y.append(1)
        else:
            y.append(-1)
    return y

### Подготовка данных для обучения: 

1. Получение гистограмм для каждого изображения из набора данных
2. Определение отклика для изображений (0 -кот; 1 - собака)
3. Разделение данных на тренировочные и тестировочные в соотношении 75/25

In [20]:
X = [extract_histogram(cv2.imread(image)) for image in imagePaths]
y = np.array(classification(imagePaths))
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size = 0.25, random_state = 19)

### Изображения для которых необходимо определить отклик 

In [37]:
SAMPLE_LIST = ["test_task\\test\\cat.1002.jpg",
               "test_task\\test\\cat.1018.jpg",
               "test_task\\test\\dog.1022.jpg",
               "test_task\\test\\dog.1035.jpg"]

### Обучение классификатора LinearSVC

In [25]:
clf = LinearSVC(C=0.6,random_state=19)
clf.fit(X_train, y_train)

LinearSVC(C=0.6, random_state=19)

### Расчет точности (precision), полноты (recall) и значение метрики F1 

In [42]:
Test_sample = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[3]))).reshape(1,-1)
print(Test_sample.shape)
print(clf.predict(Test_sample))
y_predicted = np.array(clf.predict(X_test))
precision = precision_score(y_test, y_predicted)
print("Precision = ",precision)
recall = recall_score(y_test, y_predicted)
print("Recall = ", recall)
F1 = metrics.f1_score(y_test, y_predicted, average = 'weighted')
print('F1 = ', F1)

(1, 512)
[0]
Precision =  0.7368421052631579
Recall =  0.4794520547945205
F1 =  0.5930291738092567


### Интересуемые коэффициенты построенной гиперплоскости 

In [43]:
print("11= ",clf.coef_[0][10]," 13 = ",clf.coef_[0][12], " 318 = ",clf.coef_[0][317])

11=  -0.4201340427108207  13 =  -0.08045077402199619  318 =  -0.09495677350239234


### Определение классов для определенных ранее изображений

In [38]:
Test_sample = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[0]))).reshape(1,-1)
print(clf.predict(Test_sample))
Test_sample1 = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[1]))).reshape(1,-1)
print(clf.predict(Test_sample1))
Test_sample2 = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[2]))).reshape(1,-1)
print(clf.predict(Test_sample2))
Test_sample3 = np.array(extract_histogram(cv2.imread(SAMPLE_LIST[3]))).reshape(1,-1)
print(clf.predict(Test_sample3))


[0]
[0]
[0]
[0]
